In [1]:
from prophet import Prophet
m = Prophet()

In [2]:
import pandas as pd

In [3]:
forecast_sample = pd.read_csv('data/forecast_sample.csv')

In [4]:
list(forecast_sample)

['BranchId',
 'CountryCode',
 'CountryName',
 'ProductId',
 'Currency',
 'EffectiveDate',
 'Demand',
 'PlannedCashOutOrderCount',
 'AvgPlannedCashOutOrderAmount',
 'ActualCashOutOrderAmount',
 'AvgActualCashOutOrderAmount',
 'DayOfWeekName']

In [5]:
forecast_sample.dtypes

BranchId                          int64
CountryCode                      object
CountryName                      object
ProductId                         int64
Currency                         object
EffectiveDate                    object
Demand                          float64
PlannedCashOutOrderCount        float64
AvgPlannedCashOutOrderAmount    float64
ActualCashOutOrderAmount        float64
AvgActualCashOutOrderAmount     float64
DayOfWeekName                    object
dtype: object

In [6]:
forecast_sample.rename(columns={'EffectiveDate':'ds', 'Demand':'y'}, inplace=True)

In [ ]:
from prophet import Prophet

# Initialize the model
model = Prophet()

# Add additional regressors
model.add_regressor('PlannedCashOutOrderCount')
model.add_regressor('AvgPlannedCashOutOrderAmount')
model.add_regressor('ActualCashOutOrderAmount')
model.add_regressor('AvgActualCashOutOrderAmount')

# Fit the model
model.fit(forecast_sample)


In [ ]:
future = model.make_future_dataframe(periods=7)  # Forecasting 7 days into the future

# Include the additional regressors in the future DataFrame
future['PlannedCashOutOrderCount'] = forecast_sample['PlannedCashOutOrderCount']
future['AvgPlannedCashOutOrderAmount'] = forecast_sample['AvgPlannedCashOutOrderAmount']
future['ActualCashOutOrderAmount'] = forecast_sample['ActualCashOutOrderAmount']
future['AvgActualCashOutOrderAmount'] = forecast_sample['AvgActualCashOutOrderAmount']

In [ ]:
forecast = model.predict(future)

In [ ]:
fig = model.plot(forecast)
fig.show()


In [ ]:
from prophet.diagnostics import cross_validation, performance_metrics

df_cv = cross_validation(model, initial='365 days', period='180 days', horizon='7 days')
df_p = performance_metrics(df_cv)
print(df_p)

In [ ]:
m.fit(forecast_sample)  # df is a pandas.DataFrame with 'y' and 'ds' columns
future = m.make_future_dataframe(periods=7)
m.predict(future)

In [ ]:
forecast_sample = pd.read_csv('data/forecast_sample.csv')
forecast_sample.rename(columns={'EffectiveDate':'ds', 'Demand':'y'}, inplace=True)
df = forecast_sample.copy()

df['ds'] = pd.to_datetime(df['ds'])

df = df.sort_values(by='ds')

df.tail()

In [17]:
import pandas as pd
from prophet import Prophet

# Load the data
forecast_sample = pd.read_csv('data/forecast_sample.csv')
forecast_sample.rename(columns={'EffectiveDate':'ds', 'Demand':'y'}, inplace=True)
df = forecast_sample.copy()

# Convert 'ds' column to datetime
df['ds'] = pd.to_datetime(df['ds'])

# Sort the DataFrame by date
df = df.sort_values(by='ds')

# Display the last few rows of the DataFrame
print(df.tail())

# Convert categorical variables to dummy variables
df = pd.get_dummies(df, columns=['BranchId', 'ProductId', 'Currency'], drop_first=True)

# Define the split point
split_date = '2024-01-01'  # Change this to your desired split date
train = df[df['ds'] <= split_date]
test = df[df['ds'] > split_date]

# Initialize the Prophet model
model = Prophet()

# Add additional regressors
model.add_regressor('PlannedCashOutOrderCount')
model.add_regressor('AvgPlannedCashOutOrderAmount')
model.add_regressor('ActualCashOutOrderAmount')
model.add_regressor('AvgActualCashOutOrderAmount')

# Add dummy regressors for categorical variables
for col in df.columns:
    if 'BranchId_' in col or 'ProductId_' in col or 'Currency_' in col:
        model.add_regressor(col)

# Fit the model on the training set
model.fit(train)

# Make future DataFrame for prediction
future = model.make_future_dataframe(periods=7)

# Add regressors to future DataFrame
# future = pd.get_dummies(future, columns=['BranchId', 'ProductId', 'Currency'], drop_first=True)
future = future.merge(df.drop(columns=['y']), on='ds', how='left')

# Make predictions
forecast = model.predict(future)

# Display the forecast
print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail())


08:14:22 - cmdstanpy - INFO - Chain [1] start processing


      BranchId CountryCode CountryName  ProductId Currency         ds  \
2971       246          ZM      Zambia        643      USD 2024-05-12   
742         21          ZM      Zambia         72      ZMW 2024-05-12   
4457       274          ZM      Zambia        896      USD 2024-05-12   
3714       246          ZM      Zambia        644      ZMW 2024-05-12   
5200       274          ZM      Zambia        895      ZMW 2024-05-12   

            y  PlannedCashOutOrderCount  AvgPlannedCashOutOrderAmount  \
2971   4955.0                       6.0                      360.8333   
742   54800.0                      27.0                     1191.5185   
4457  11155.0                     252.0                      113.3929   
3714  59860.0                      41.0                      428.0488   
5200  31656.0                     272.0                      246.8015   

      ActualCashOutOrderAmount  AvgActualCashOutOrderAmount DayOfWeekName  
2971                    4955.0                

08:14:22 - cmdstanpy - INFO - Chain [1] done processing


             ds           yhat    yhat_lower     yhat_upper
4321 2024-01-08     271.779279  -8679.922249    9205.913803
4322 2024-01-08   25663.583483  17902.891458   34572.387495
4323 2024-01-08   80947.461332  72297.025820   89511.539079
4324 2024-01-08  102561.262886  94495.396941  111210.306388
4325 2024-01-08   23314.198513  15193.892658   32249.867252


In [19]:
print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(30))

             ds           yhat     yhat_lower     yhat_upper
4296 2024-01-04   10482.870799    1657.899833   18443.757438
4297 2024-01-04  163449.171486  154831.842440  171951.159869
4298 2024-01-05   22406.748034   14065.699656   31368.311002
4299 2024-01-05  102797.950582   94712.558225  110504.373423
4300 2024-01-05     -41.115887   -8048.001165    8938.729618
4301 2024-01-05   18405.288921    9739.884992   27293.331954
4302 2024-01-05    7631.955881    -915.910163   16025.546908
4303 2024-01-05  101109.909001   93233.751784  109310.966625
4304 2024-01-05  154349.376570  146111.678853  162802.505687
4305 2024-01-06  112374.894026  103352.617377  121295.450929
4306 2024-01-06  114280.364922  105794.218265  122788.125063
4307 2024-01-06   19335.915551   11136.374801   27479.353019
4308 2024-01-06   18116.045268    9776.664985   27105.172916
4309 2024-01-06   13713.606556    4968.258929   22039.695003
4310 2024-01-06   32745.943945   23632.436747   41418.264444
4311 2024-01-06    1111.

In [20]:
test

,CountryCode,CountryName,ds,y,PlannedCashOutOrderCount,AvgPlannedCashOutOrderAmount,ActualCashOutOrderAmount,AvgActualCashOutOrderAmount,DayOfWeekName,BranchId_244,BranchId_246,BranchId_274,ProductId_641,ProductId_642,ProductId_643,ProductId_644,ProductId_895,ProductId_896,Currency_ZMW
2097,ZM,Zambia,2024-01-02,8752.0,24.0,295.8333,8752.0,397.8181,Tuesday,True,False,False,True,False,False,False,False,False,True
4326,ZM,Zambia,2024-01-02,8300.0,87.0,130.4023,8300.0,90.2173,Tuesday,False,False,True,False,False,False,False,False,True,False
2840,ZM,Zambia,2024-01-02,6425.0,210.0,70.2143,6425.0,66.9270,Tuesday,False,True,False,False,False,True,False,False,False,False
3583,ZM,Zambia,2024-01-02,48815.0,142.0,508.4859,47375.0,707.0895,Tuesday,False,True,False,False,False,False,True,False,False,True
5069,ZM,Zambia,2024-01-02,38424.0,87.0,889.8161,34924.0,623.6428,Tuesday,False,False,True,False,False,False,False,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2971,ZM,Zambia,2024-05-12,4955.0,6.0,360.8333,4955.0,381.1538,Sunday,False,True,False,False,False,True,False,False,False,False
742,ZM,Zambia,2024-05-12,54800.0,27.0,1191.5185,54800.0,1442.1052,Sunday,False,False,False,False,False,False,False,False,False,True
4457,ZM,Zambia,2024-05-12,11155.0,252.0,113.3929,11155.0,122.5824,Sunday,False,False,True,False,False,False,False,False,True,False
3714,ZM,Zambia,2024-05-12,59860.0,41.0,428.0488,56420.0,989.8245,Sunday,False,True,False,False,False,False,True,False,False,True


In [21]:
test_forecast = model.predict(test)

In [22]:
test_forecast

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,ActualCashOutOrderAmount,ActualCashOutOrderAmount_lower,ActualCashOutOrderAmount_upper,AvgActualCashOutOrderAmount,...,extra_regressors_additive,extra_regressors_additive_lower,extra_regressors_additive_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2024-01-02,36417.670301,1186.771479,18497.610928,36417.670301,36417.670301,-27888.317096,-27888.317096,-27888.317096,215.217028,...,-26254.160184,-26254.160184,-26254.160184,-92.044632,-92.044632,-92.044632,0.0,0.0,0.0,10071.465485
1,2024-01-02,36417.670301,-346.647602,16941.231026,36417.670301,36417.670301,-28340.799794,-28340.799794,-28340.799794,374.060697,...,-28277.030647,-28277.030647,-28277.030647,-92.044632,-92.044632,-92.044632,0.0,0.0,0.0,8048.595022
2,2024-01-02,36417.670301,-7684.929438,10018.900631,36417.670301,36417.670301,-30217.802136,-30217.802136,-30217.802136,386.087703,...,-34753.524932,-34753.524932,-34753.524932,-92.044632,-92.044632,-92.044632,0.0,0.0,0.0,1572.100738
3,2024-01-02,36417.670301,37747.213633,54474.137432,36417.670301,36417.670301,10775.929018,10775.929018,10775.929018,55.510669,...,9709.108651,9709.108651,9709.108651,-92.044632,-92.044632,-92.044632,0.0,0.0,0.0,46034.734320
4,2024-01-02,36417.670301,28306.592561,45361.416978,36417.670301,36417.670301,-1688.367602,-1688.367602,-1688.367602,98.602169,...,569.861950,569.861950,569.861950,-92.044632,-92.044632,-92.044632,0.0,0.0,0.0,36895.487619
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
919,2024-05-12,36735.307650,-5019.068483,11945.084002,36532.851991,36956.400909,-34313.170979,-34313.170979,-34313.170979,311.078778,...,-34304.053839,-34304.053839,-34304.053839,532.003050,532.003050,532.003050,0.0,0.0,0.0,2963.256861
920,2024-05-12,36735.307650,-7263.853698,9546.365448,36532.519829,36957.191641,-31689.371972,-31689.371972,-31689.371972,223.822398,...,-35810.716145,-35810.716145,-35810.716145,532.003050,532.003050,532.003050,0.0,0.0,0.0,1456.594555
921,2024-05-12,36735.307650,47918.405038,66023.585361,36532.187666,36957.749206,18208.858293,18208.858293,18208.858293,-324.048132,...,19330.995943,19330.995943,19330.995943,532.003050,532.003050,532.003050,0.0,0.0,0.0,56598.306643
922,2024-05-12,36735.307650,3142.854359,19825.837742,36531.855504,36958.212291,-25482.750894,-25482.750894,-25482.750894,357.347505,...,-25938.712069,-25938.712069,-25938.712069,532.003050,532.003050,532.003050,0.0,0.0,0.0,11328.598631


In [23]:
df

,CountryCode,CountryName,ds,y,PlannedCashOutOrderCount,AvgPlannedCashOutOrderAmount,ActualCashOutOrderAmount,AvgActualCashOutOrderAmount,DayOfWeekName,BranchId_244,BranchId_246,BranchId_274,ProductId_641,ProductId_642,ProductId_643,ProductId_644,ProductId_895,ProductId_896,Currency_ZMW
0,ZM,Zambia,2022-05-01,38590.0,115.0,472.2609,38590.0,665.3448,Sunday,False,False,False,False,False,False,False,False,False,True
2972,ZM,Zambia,2022-05-01,12100.0,34.0,592.3529,12100.0,1728.5714,Sunday,False,True,False,False,False,False,True,False,False,True
2229,ZM,Zambia,2022-05-01,-200.0,4.0,50.0000,100.0,50.0000,Sunday,False,True,False,False,False,True,False,False,False,False
3715,ZM,Zambia,2022-05-01,285.0,2.0,55.0000,285.0,71.2500,Sunday,False,False,True,False,False,False,False,False,True,False
1486,ZM,Zambia,2022-05-01,14360.0,15.0,574.6667,10480.0,551.5789,Sunday,True,False,False,True,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2971,ZM,Zambia,2024-05-12,4955.0,6.0,360.8333,4955.0,381.1538,Sunday,False,True,False,False,False,True,False,False,False,False
742,ZM,Zambia,2024-05-12,54800.0,27.0,1191.5185,54800.0,1442.1052,Sunday,False,False,False,False,False,False,False,False,False,True
4457,ZM,Zambia,2024-05-12,11155.0,252.0,113.3929,11155.0,122.5824,Sunday,False,False,True,False,False,False,False,False,True,False
3714,ZM,Zambia,2024-05-12,59860.0,41.0,428.0488,56420.0,989.8245,Sunday,False,True,False,False,False,False,True,False,False,True
